#### SimpleTransformers
Few resources :
1. https://www.philschmid.de/bert-text-classification-in-a-different-language
2. https://github.com/ThilinaRajapakse/simpletransformers

In [1]:
#Uncomment if not installed
#pip install simpletransformers

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
layer 0.10.3081287781 requires aiohttp<3.8.0,>=3.7.3, but you have aiohttp 3.8.3 which is incompatible.
layer 0.10.3081287781 requires pandas==1.3.5, but you have pandas 1.4.4 which is incompatible.


  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=622bb6fa2cd58ef146d2bdcba62c733266afed713034a58e7c64aa23e8d073fe
  Stored in directory: c:\users\chopr.000\appdata\local\pip\cache\wheels\b7\0a\67\ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=4ba0a460efb5346522427837914151a0c50a07519e98fba268a00222620620ed
  Stored in directory: c:\users\chopr.000\appdata\local\pip\cache\wheels\e2\a5\92\2c80d1928733611c2747a9820e1324a6835524d9411510c142
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=bf2fb61e9b4f40845c233744b12c19a45d6efecf40b12d9aaae5c0d6d3ff9b31
  Stored in directory: c:\users\chopr.000\appdata\local\pip\cache\wheels\2d\f0\a8\1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built pathtools seqeval validators
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uni

In [19]:
#curl https://projects.fzai.h-da.de/iggsa/wp-content/uploads/2019/08/germeval2019GoldLabelsSubtask1_2.txt
#curl https://projects.fzai.h-da.de/iggsa/wp-content/uploads/2019/09/germeval2019.training_subtask1_2_korrigiert.txt

##The above two downloads have been moved to the following two places

#!curl https://fz.h-da.de/iggsa//wp-content/uploads/2019/08/germeval2019GoldLabelsSubtask1_2.txt --output germeval2019.GoldLabelsSubtask1_2.txt
#!curl https://fz.h-da.de/iggsa//wp-content/uploads/2019/09/germeval2019.training_subtask1_2_korrigiert.txt --output germeval2019.training_subtask1_2_korrigiert.txt
    
!curl https://fz.h-da.de/fileadmin/user_upload/germeval2019GoldLabelsSubtask1_2.txt --output germeval2019GoldLabelsSubtask1_2.txt
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 15  531k   15 81920    0     0  49825      0  0:00:10  0:00:01  0:00:09 49829
100  531k  100  531k    0     0   236k      0  0:00:02  0:00:02 --:--:--  236k


##### Data Cleansing

In [58]:

import pandas as pd

df1=pd.read_csv('germeval2019GoldLabelsSubtask1_2.txt',sep='\t', lineterminator='\n',encoding='utf8',names=["tweet", "task1", "task2"])

'''
# to clear trailing '\r' from task2 column
for i,elem in enumerate(df1['task2']):
    df1['task2'][i]=df1['task2'][i].replace('\r','')
'''

df1['task2']=df1['task2'].apply(lambda x: x.replace('\r',''))

#class_list=list(set(df1['task2']))


class_list=['OTHER', 'ABUSE', 'PROFANITY', 'INSULT']


df1['pred_class']=df1['task2'].apply(lambda x:class_list.index(x))

df = df1[['tweet','pred_class']]

df.head()



,tweet,pred_class
0,@JanZimmHHB @mopo Komisch das die Realitätsver...,3
1,@faznet @Gruene_Europa @SPDEuropa @CDU CDU ste...,1
2,"@DLFNachrichten Die Gesichter, Namen, Religion...",0
3,@welt Wie verwirrt muss man sein um sich zu we...,1
4,@hacker_1991 @torben_braga Weil die AfD den Fe...,1


In [97]:
class_list

['OTHER', 'ABUSE', 'PROFANITY', 'INSULT']

##### Train Test Split


In [73]:
from sklearn.model_selection import train_test_split
df_train,df_test=train_test_split(df,test_size=0.1)

In [72]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={
            "reprocess_input_data": True,
            "fp16":False,
            #"num_train_epochs": 4
            "num_train_epochs": 1
            }

# Create a ClassificationModel
model = ClassificationModel(
                            "bert", "distilbert-base-german-cased",
                            num_labels=4,
                            use_cuda=False,
                            args=train_args
                            )


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing BertForSequenceClassification: ['distilbert.transformer.layer.3.ffn.lin1.bias', 'distilbert.transformer.layer.3.sa_layer_norm.bias', 'distilbert.transformer.layer.3.attention.v_lin.bias', 'distilbert.transformer.layer.1.attention.v_lin.weight', 'distilbert.transformer.layer.2.ffn.lin2.weight', 'distilbert.transformer.layer.3.attention.k_lin.bias', 'distilbert.transformer.layer.1.output_layer_norm.weight', 'distilbert.transformer.layer.4.attention.k_lin.weight', 'distilbert.transformer.layer.4.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.1.attention.out_lin.bias', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.4.ffn.lin2.bias', 'distilbert.transformer.layer.5.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.3.attention.out_lin.bias', 'distil

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


##### Training the model

In [77]:
model.train_model(df_train)

C:\ProgramData\Anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2727 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/341 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/341 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/341 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/341 [00:00<?, ?it/s]

(1364, 0.8648303075313918)

#### Evaluation

In [81]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(df_test, f1=f1_multiclass, acc=accuracy_score)



C:\ProgramData\Anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/304 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

In [94]:
import os
import tarfile

def pack_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

# run the function
pack_model('outputs','germeval-distilbert-german')
#save_model('outputs','germeval-distilbert-german')

In [96]:
def unpack_model(model_name=''):
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

unpack_model('germeval-distilbert-german')